In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1000 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/kaggle/input/r-32-a-64-b8/r-32-a-64-warm-50-b-8-acc-4/checkpoint-800", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Unsloth 2024.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# print and see dataset
dataset

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 1000000
    })
    test: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 10000
    })
})

In [5]:

prompt = """You are a math grader tasked with evaluating whether a given answer to a math question is correct or not. Respond with 'True' if the answer is correct and 'False' if it is incorrect.

Below are the Question, the given Answer, and the Explanation of the Answer.

### Question:
{}

### Answer:
{}

### Explanation:
{}

### It’s very important to grade the Answer accurately, so you must
1.  Carefully read and understand the Question.
2.  Review the given Answer and compare it against the Explanation provided.
3.  If the Explanation correctly justifies the Answer, respond with 'True'.
4.  If the Explanation is incorrect or does not logically support the Answer, respond with 'False'.

### Grading ('True' or 'False'):
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    answer = examples["answer"]
    explanation = examples["solution"]
    output = examples["is_correct"]
    texts = []
    for q, a, e, o in zip(question, answer, explanation, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(q, a, e, o) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts}



TRAIN

In [6]:
#train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

In [7]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported

# training_args = TrainingArguments(
#         per_device_train_batch_size = 8,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         #num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 300,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "outputs",
#         report_to = "none", # Use this for WandB etc
#     )

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = train_dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 4,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = training_args
# )

In [8]:
# trainer_stats = trainer.train()

In [9]:
# model.save_pretrained("lora_model2") # Local saving
# tokenizer.save_pretrained("lora_model2")

INFERENCE

In [10]:
# Sample inferene data point
test_dataset = dataset['test']

sample_ques = test_dataset['question']
sample_ans = test_dataset['answer']
sample_sol = test_dataset['solution']


In [11]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [12]:
torch.cuda.empty_cache()

In [13]:
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
sol = []
# Prepare your input prompts
input_prompts = []
for i in range(len(sample_ques)):
    input = prompt.format(
        sample_ques[i],  # question
        sample_ans[i],   # given answer
        sample_sol[i],   # explanation
        "",  # output - leave this blank for generation
    )
    input_prompts.append(input)

chunk_size = 20 # Divide your dataset into smaller chunks
for i in range(0, len(input_prompts), chunk_size):
    chunk = input_prompts[i:i + chunk_size]
    inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=False, max_length= max_seq_length).to("cuda")
    input_shape = inputs['input_ids'].shape
    input_token_len = input_shape[1] # 1 because of batch
    # Tokenize and run inference for the chunk

    outputs = model.generate(**inputs, max_new_tokens=1,use_cache=True)
    response = tokenizer.batch_decode(outputs[:, input_token_len:], skip_special_tokens=True)
    print(i)
    sol = sol + response

    # Clear cache after each chunk to free up memory
    torch.cuda.empty_cache()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
4

In [14]:
len(sol)

10000

In [15]:
import pandas as pd

In [16]:
ID = [i for i in range(len(sol))]
is_correct = [s == 'True' for s in sol]
dict = {'ID': ID, 'is_correct': is_correct}
df = pd.DataFrame(dict)

df.to_csv('r32-a64-b8.csv', index=False)
